In [4]:
import json
import re
import emoji
import glob
import pandas as pd
import nltk
import random

In [8]:

def clean_conversations_format(conversations):
    
    new_conversations = []
    regex = " {2,}" # remove multiple space
    regex_2 = "\n {1,}\n" # remove irregular \n
    regex_3 = "\n {1,}"
    regex_4 = "[$/<>\"+\\\)(}{]" # remove the simble
    regex_5 = "\n{2,}" # change \n\n to \n and use \n as separator
    regex_6 = "\t{1,}"
    for c in conversations:
        multiple_s = re.findall(regex, c)
        c = re.sub(regex," ", c) # change to one space
        c = re.sub(regex_2, "\n\n",c) #
        c = re.sub(regex_3," ", c) # change to one space
        c = re.sub(regex_4, "", c)
        c = re.sub(regex_5, "\n",c)
        c = re.sub(regex_6, "", c)
        emoji.replace_emoji(c)
        c = emoji.replace_emoji(c)

        # # test effects
        # if "No, everything was fine, but my knee gave me a hard time this time." in c:
        #     print(c)
        #     # break
        new_conversations.append(c)

    return new_conversations


In [9]:
def remove_role(new_conversations):
    list_con_utterances= [] 
    count_utterances = 0
    for con in new_conversations:
            utterances = con.split("\n")
            remove_head_u = []
            for u in utterances:
                if u.startswith("F: "):
                    u = u.replace("F: ", "")
                elif u.startswith("F:"):
                    u = u.replace("F:","")
                elif u.startswith("P: "):
                    u = u.replace("P: ", "")
                elif u.startswith("P:"):
                    u = u.replace("P:", "")
                if len(u)>1:
                    remove_head_u.append(u)
            list_con_utterances.append(remove_head_u)
            count_utterances += len(remove_head_u)
            list_con_utterances.append(["-"*20])
    list_u_sep = []
    for l in list_con_utterances:
        for s in l:
            list_u_sep.append(s)
    return list_u_sep, count_utterances   

In [29]:
dir = '../response_data/'
count = 0
count_all_utterances = 0 
n_file = 0
files = glob.glob(f"{dir}raw_conversations/*.json")
for f in files:
    with open (f) as infile:
        data = json.load(infile)
# create a dict with cleantext
    cleandata = dict()
    for key, value in data.items():
        category = key
        conversations = value
        new_conversations = clean_conversations_format(conversations) # list of str
        list_u_sep, count_utterances = remove_role(new_conversations)
        cleandata[category] = new_conversations
        
        # wirte to tsv file
        df = pd.DataFrame({'col':list_u_sep})
        df.to_csv(f"{dir}clean_conversations/{category}_clean.tsv",sep="\t",index=False,header=["utterances"])
        
        # write to json file
        with open(f"{dir}clean_conversations/{category}_clean.json", "w", encoding="utf-8") as jfile:
            json.dump(cleandata, jfile, ensure_ascii=False, indent=4)
        num_c = len(new_conversations)
        print(category, num_c)
        count += num_c
        count_all_utterances += count_utterances
        n_file += 1
print(f"finished {n_file} files, total: {count} conversations, {count_all_utterances} utterances, ")

    

mobility 120
general tasks and demands 120
communication 120
major life areas 120
domestic life 120
learning and applying knowledge 120
interpersonal interactions and relationships 120
self-care 140
community, social and civic life 100
finished 9 files, total: 1080 conversations, 8464 utterances, 


In [44]:
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
count_token = 0
files_tokens = glob.glob("./clean_conversations/*.tsv")
for f_token in files_tokens:
    df_token = pd.read_csv(f"{f_token}", sep="\t")
    for u in df_token["utterances"]:
        tokens = tokenizer.tokenize(u)
        count_t = len(tokens)
        count_token += count_t
print("totoal tokens of all conversations are", count_token)


totoal tokens of all conversations are 121046
